In [77]:
import os
import json

from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np

In [78]:
os.chdir("/home/u5u/kdeng.u5u/spatial-reasoning-of-LMs")

[ ] introduce dataset stat table, and calculate weighted average

In [79]:
col_name_map = {
    "dataset": "Dataset",
    "model": "Model",
    "strategy": "Strategy",
    "split": "DoF",

    "accuracy": "Acc.",
    "f1_score": "F1",
}

useful_map = {
    "gpt-4o": "GPT-4o",
    "Qwen2.5-VL-7B-Instruct": "Qwen-7B",
    "Qwen2.5-VL-32B-Instruct": "Qwen-32B",
    "Qwen2.5-VL-72B-Instruct": "Qwen-72B",

    "sift": "SIFT",
    "loftr": "LoFTR",

    "7-scenes": "7 Scenes",
    "scannet": "ScanNet",
    "scannetpp": "ScanNet++",

    "theta": "Pitch",
    "phi": "Yaw",
    "psi": "Roll",
    "tx": "L/R",
    "ty": "U/D",
    "tz": "F/B",

    # "dataset-prior-hint": "w/ dataset prior",
    # "CoT-hint": "w/ dataset prior + CoT",
    # "VoT-hint": "w/ dataset prior + VoT",

    "zero-shot": "ZS",
    "dataset-prior-hint": "w/ DP",
    "CoT-hint": "DP+CoT",
    "VoT-hint": "DP+VoT",

    np.nan: "—", 
    None: "—",
}


In [80]:
dataset_order = ["7 Scenes", "ScanNet", "ScanNet++"]
model_order = ["SIFT", "LoFTR", "Qwen-7B", "Qwen-32B", "Qwen-72B", "GPT-4o"]
dof_order = ["Pitch", "Yaw", "Roll", "U/D", "L/R", "F/B"]
# strategy_order = ["zero-shot", "w/ dataset prior", "w/ dataset prior + CoT", "w/ dataset prior + VoT"]
strategy_order = ["ZS", "w/ DP", "DP+CoT", "DP+VoT"]

### Useful functions for making table

In [81]:
def _read_c1_vlm(data_dir: str) -> pd.DataFrame:
    data_dir = Path(data_dir)
    data = []
    for dataset in data_dir.iterdir():
        if dataset.is_dir():
            for model in dataset.iterdir():
                if model.is_dir():
                    for stratrgy in model.iterdir():
                        if stratrgy.is_dir():
                            for split in stratrgy.iterdir():
                                if split.is_dir():
                                    try: 
                                        metrics_path = split / "metrics" / "metrics.json"
                                        with open(metrics_path, "r") as f:
                                            metrics = json.load(f)
                                        
                                        data.append({
                                            "dataset": dataset.name,
                                            "model": model.name,
                                            "strategy": stratrgy.name,
                                            "split": split.name,
                                            "accuracy": metrics.get("accuracy", None),
                                            "f1_score": metrics.get("f1_score", None),
                                            "precision": metrics.get("precision", None),
                                            "recall": metrics.get("recall", None)
                                        })
                                    
                                    except Exception as e:
                                        print(f"Error processing data: {e}, dataset: {dataset}, model: {model}, strategy: {stratrgy}, split: {split}")

    df = pd.DataFrame(data)
    return df


def _read_c1_cv(data_dir: str) -> pd.DataFrame:
    data_dir = Path(data_dir)
    data = []
    for dataset in data_dir.iterdir():
        if dataset.is_dir():
            for model in dataset.iterdir():
                if model.is_dir():
                    for split in model.iterdir():
                        if split.is_dir():

                            try: 
                                metrics_path = split / "metrics" / "metrics.json"
                                with open(metrics_path, "r") as f:
                                    metrics = json.load(f)
                                
                                data.append({
                                    "dataset": dataset.name,
                                    "model": model.name,
                                    "strategy": None,
                                    "split": split.name,
                                    "accuracy": metrics.get("accuracy", None),
                                    "f1_score": metrics.get("f1_score", None),
                                    "precision": metrics.get("precision", None),
                                    "recall": metrics.get("recall", None)
                                })
                            
                            except Exception as e:
                                print(f"Error processing data: {e}, dataset: {dataset}, model: {model}, split: {split}")
                                
    return pd.DataFrame(data)


def _rename_table(df: pd.DataFrame) -> pd.DataFrame:
    df = df.rename(columns=col_name_map)
    df = df.replace(useful_map)
    return df


def _set_factor_as_ordered_cate(df: pd.DataFrame, col: str, order: list) -> pd.DataFrame:
    """
    Set a column as ordered categorical with a specific order.
    """
    df[col] = pd.Categorical(df[col], categories=order, ordered=True)
    return df


def _weighted_avg(row, df: pd.DataFrame, counting_table: pd.DataFrame):
    acc_cols = [col for col in df.columns if col[1] == 'Acc.']
    f1_cols = [col for col in df.columns if col[1] == 'F1']

    # get the row index to see which data
    weights = counting_table.loc[row.name[0], :]

    # calculate the weighted average
    acc_weighted_avg = sum(row[col] * weights.get(col[0], 1) for col in acc_cols) / sum(weights.get(col[0], 1) for col in acc_cols)
    f1_weighted_avg = sum(row[col] * weights.get(col[0], 1) for col in f1_cols) / sum(weights.get(col[0], 1) for col in f1_cols)
    
    return pd.Series({
        ('Avg', 'Acc.'): acc_weighted_avg,
        ('Avg', 'F1'): f1_weighted_avg
    })

    # df[('Avg', 'Acc.')] = sum(df[col] * counting_table.get(col[0], 1) for col in acc_cols) / sum(counting_table.get(col[0], 1) for col in acc_cols)
    # df[('Avg', 'F1')] = sum(df[col] * counting_table.get(col[0], 1) for col in f1_cols) / sum(counting_table.get(col[0], 1) for col in f1_cols)


def _cal_avg_of_acc_f1(df: pd.DataFrame, **kwargs) -> pd.DataFrame:
    """
    Calculate the average of accuracy and F1 score for each group.
    """
    count = kwargs.get("count")

    if count is not None: # count as weigt for each col
        avg_cols = df.apply(lambda row: _weighted_avg(row, df, count), axis=1)
        df = pd.concat([df, avg_cols], axis=1)

    else:
        acc_cols = [col for col in df.columns if col[1] == 'Acc.']
        f1_cols = [col for col in df.columns if col[1] == 'F1']

        df[('Avg', 'Acc.')] = df[acc_cols].mean(axis=1)
        df[('Avg', 'F1')] = df[f1_cols].mean(axis=1)

    return df

# C1 without trap option

In [82]:
c1_without_trap = _read_c1_vlm("/home/u5u/kdeng.u5u/spatial-reasoning-of-LMs/result/final-table-wo-trap/single-dof-cls")
c1_without_trap["is_trap"] = False
c1_without_trap["is_shuffle"] = True
c1_without_trap = _rename_table(c1_without_trap)

c1_without_trap = _set_factor_as_ordered_cate(c1_without_trap, "Dataset", dataset_order)
c1_without_trap = _set_factor_as_ordered_cate(c1_without_trap, "Model", model_order)
c1_without_trap = _set_factor_as_ordered_cate(c1_without_trap, "DoF", dof_order)
c1_without_trap = _set_factor_as_ordered_cate(c1_without_trap, "Strategy", strategy_order)

In [83]:
counting_table = pd.read_csv("/home/u5u/kdeng.u5u/spatial-reasoning-of-LMs/result/csv-tables/c1-counting-table.csv")

In [84]:
counting_table.set_index(["Dataset"], inplace=True)
counting_table

,Pitch,Yaw,Roll,U/D,L/R,F/B
Dataset,,,,,,
7 Scenes,58.0,85.0,87.0,32.0,10.0,18.0
ScanNet,122.0,143.0,34.0,48.0,15.0,19.0
ScanNet++,146.0,262.0,2.0,63.0,4.0,16.0


In [85]:
pivot = c1_without_trap.pivot_table(
    index=["Dataset", "Model", "Strategy"],
    columns="DoF",
    values=["Acc.", "F1"],
)
swap = pivot.swaplevel(0, 1, axis=1).sort_index(axis=1)
swap.columns.names = [None, None]
swap = _cal_avg_of_acc_f1(swap, count=counting_table).round(3)
# swap = _cal_avg_of_acc_f1(swap).round(3)

swap.to_csv("result/csv-tables/c1-wo-trap.csv")
swap

/local/user/1483801110/ipykernel_110261/964906686.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = c1_without_trap.pivot_table(


Pitch           Yaw          Roll           U/D  \
                              Acc.     F1   Acc.     F1   Acc.     F1   Acc.   
Dataset   Model    Strategy                                                    
7 Scenes  Qwen-7B  ZS        0.655  0.672  0.550  0.498  0.450  0.500  0.700   
                   w/ DP     0.741  0.659  0.550  0.512  0.567  0.567  0.500   
                   DP+CoT    0.310  0.168  0.617  0.534  0.400  0.355  1.000   
                   DP+VoT    0.310  0.168  0.450  0.449  0.583  0.471  0.600   
          Qwen-32B ZS        0.879  0.878  0.817  0.814  0.450  0.376  0.900   
                   w/ DP     0.862  0.852  0.783  0.776  0.483  0.473  0.700   
                   DP+CoT    0.741  0.718  0.833  0.833  0.467  0.485  0.400   
                   DP+VoT    0.741  0.718  0.767  0.766  0.550  0.549  0.700   
          Qwen-72B ZS        0.845  0.883  0.817  0.816  0.583  0.637  0.900   
                   w/ DP     0.793  0.795  0.850  0.850  0.583  0.561  0.700   
                   DP+CoT    0.310  0.300  0.433  0.348  0.383  0.357  0.500   
                   DP+VoT    0.500  0.521  0.600  0.584  0.550  0.536  0.600   
          GPT-4o   ZS        0.845  0.850  0.900  0.900  0.450  0.452  1.000   
                   w/ DP     0.879  0.884  0.800  0.800  0.450  0.449  0.700   
                   DP+CoT    0.638  0.617  0.467  0.416  0.467  0.467  0.700   
                   DP+VoT    0.259  0.253  0.583  0.578  0.550  0.546  0.400   
ScanNet   Qwen-7B  ZS        0.900  0.908  0.683  0.686  0.559  0.585  0.733   
                   w/ DP     0.700  0.652  0.717  0.709  0.588  0.591  0.667   
                   DP+CoT    0.433  0.297  0.367  0.327  0.441  0.301  0.600   
                   DP+VoT    0.417  0.330  0.517  0.505  0.412  0.240  0.467   
          Qwen-32B ZS        0.967  0.974  0.850  0.848  0.588  0.520  0.867   
                   w/ DP     0.950  0.949  0.817  0.816  0.471  0.433  0.933   
                   DP+CoT    0.850  0.841  0.717  0.716  0.588  0.559  0.667   
                   DP+VoT    0.908  0.905  0.865  0.864  0.400  0.403  0.667   
          Qwen-72B ZS        0.967  0.966  0.917  0.927  0.294  0.361  0.844   
                   w/ DP     0.983  0.983  0.870  0.869  0.412  0.416  0.733   
                   DP+CoT    0.500  0.503  0.723  0.682  0.490  0.493  0.367   
                   DP+VoT    0.867  0.868  0.731  0.701  0.485  0.454  0.667   
          GPT-4o   ZS        1.000  1.000  0.883  0.884  0.382  0.385  1.000   
                   w/ DP     1.000  1.000  0.767  0.768  0.441  0.340  1.000   
                   DP+CoT    0.583  0.582  0.617  0.594  0.500  0.488  0.667   
                   DP+VoT    0.383  0.350  0.567  0.513  0.500  0.497  0.467   
ScanNet++ Qwen-7B  ZS        0.917  0.917  0.600  0.549  0.500  0.667  0.500   
                   w/ DP     0.567  0.427  0.767  0.761  0.500  0.667  0.750   
                   DP+CoT    0.567  0.496  0.517  0.379  0.000  0.000  0.500   
                   DP+VoT    0.583  0.534  0.533  0.517  0.000  0.000  0.750   
          Qwen-32B ZS        0.950  0.950  0.600  0.586  1.000  1.000  0.500   
                   w/ DP     0.883  0.880  0.700  0.685  1.000  1.000  0.750   
                   DP+CoT    0.783  0.767  0.717  0.714  0.500  0.667  0.500   
                   DP+VoT    0.883  0.880  0.667  0.666  0.500  0.667  0.750   
          Qwen-72B ZS        0.963  0.981  0.931  0.931  0.000  0.000  0.875   
                   w/ DP     0.972  0.979  0.870  0.870  0.250  0.400  1.000   
                   DP+CoT    0.487  0.490  0.548  0.453  0.750  0.857  0.500   
                   DP+VoT    0.616  0.572  0.636  0.624  0.167  0.286  0.875   
          GPT-4o   ZS        0.967  0.967  0.833  0.833  0.500  0.667  1.000   
                   w/ DP     0.950  0.950  0.767  0.765  0.000  0.000  1.000   
                   DP+CoT    0.567  0.565  0.550  0.495  0.000  0.000  1.000   
                   DP+VoT    

In [86]:
# print latex code
print(swap.to_latex(
    column_format="lll|cc|cc|cc|cc|cc|cc|cc",
    multirow=True,
    multicolumn=True,
    multicolumn_format="c",
    float_format="%.3f",
    escape=False,
    label="tab:c1-without-trap",
    caption="C1 without trap.",
    position="tb",
    bold_rows=True,
))

\begin{table}[tb]
\caption{C1 without trap.}
\label{tab:c1-without-trap}
\begin{tabular}{lll|cc|cc|cc|cc|cc|cc|cc}
\toprule
 &  &  & \multicolumn{2}{c}{Pitch} & \multicolumn{2}{c}{Yaw} & \multicolumn{2}{c}{Roll} & \multicolumn{2}{c}{U/D} & \multicolumn{2}{c}{L/R} & \multicolumn{2}{c}{F/B} & \multicolumn{2}{c}{Avg} \\
 &  &  & Acc. & F1 & Acc. & F1 & Acc. & F1 & Acc. & F1 & Acc. & F1 & Acc. & F1 & Acc. & F1 \\
Dataset & Model & Strategy &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{16}{*}{\textbf{7 Scenes}} & \multirow[t]{4}{*}{\textbf{Qwen-7B}} & \textbf{ZS} & 0.655 & 0.672 & 0.550 & 0.498 & 0.450 & 0.500 & 0.700 & 0.709 & 0.594 & 0.490 & 0.667 & 0.622 & 0.566 & 0.564 \\
\textbf{} & \textbf{} & \textbf{w/ DP} & 0.741 & 0.659 & 0.550 & 0.512 & 0.567 & 0.567 & 0.500 & 0.475 & 0.531 & 0.528 & 0.778 & 0.681 & 0.601 & 0.565 \\
\textbf{} & \textbf{} & \textbf{DP+CoT} & 0.310 & 0.168 & 0.617 & 0.534 & 0.400 & 0.355 & 1.000 & 1.000 & 0.438 & 0.299 & 0.500 & 0.533 & 0.519 &

# C1 with trap option

In [87]:
data_dir = Path("/home/u5u/kdeng.u5u/spatial-reasoning-of-LMs/result/final-table-w-trap/single-dof-cls")
c1_with_trap = _read_c1_vlm(data_dir)
c1_with_trap["is_trap"] = True
c1_with_trap["is_shuffle"] = True
c1_with_trap = _rename_table(c1_with_trap)

c1_with_trap = _set_factor_as_ordered_cate(c1_with_trap, "Dataset", dataset_order)
c1_with_trap = _set_factor_as_ordered_cate(c1_with_trap, "Model", model_order)
c1_with_trap = _set_factor_as_ordered_cate(c1_with_trap, "DoF", dof_order)
c1_with_trap = _set_factor_as_ordered_cate(c1_with_trap, "Strategy", strategy_order)

In [88]:
pivot = c1_with_trap.pivot_table(
    index=["Dataset", "Model", "Strategy"],
    columns="DoF",
    values=["Acc.", "F1"],
)
swap = pivot.swaplevel(0, 1, axis=1).sort_index(axis=1)
swap.columns.names = [None, None]
swap = _cal_avg_of_acc_f1(swap, count=counting_table).round(3)

swap.to_csv("result/csv-tables/c1-w-trap.csv")
swap

/local/user/1483801110/ipykernel_110261/3583586247.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = c1_with_trap.pivot_table(


Pitch           Yaw          Roll           U/D  \
                              Acc.     F1   Acc.     F1   Acc.     F1   Acc.   
Dataset   Model    Strategy                                                    
7 Scenes  Qwen-7B  ZS        0.052  0.092  0.200  0.285  0.000  0.000  0.100   
                   w/ DP     0.655  0.566  0.500  0.480  0.333  0.420  0.500   
                   DP+CoT    0.310  0.218  0.517  0.412  0.417  0.418  0.800   
                   DP+VoT    0.397  0.342  0.500  0.402  0.483  0.425  0.300   
          Qwen-32B ZS        0.207  0.325  0.578  0.687  0.100  0.151  0.400   
                   w/ DP     0.741  0.802  0.717  0.730  0.367  0.489  0.700   
                   DP+CoT    0.672  0.734  0.600  0.635  0.300  0.398  0.400   
                   DP+VoT    0.655  0.717  0.483  0.523  0.317  0.422  0.600   
          Qwen-72B ZS        0.552  0.696  0.850  0.864  0.267  0.344  0.800   
                   w/ DP     0.810  0.827  0.850  0.850  0.467  0.470  0.800   
                   DP+CoT    0.500  0.518  0.550  0.529  0.600  0.602  0.300   
                   DP+VoT    0.621  0.641  0.550  0.540  0.517  0.506  0.500   
          GPT-4o   ZS        0.759  0.781  0.817  0.816  0.467  0.456  0.800   
                   w/ DP     0.776  0.786  0.700  0.700  0.550  0.499  0.900   
                   DP+CoT    0.466  0.489  0.517  0.494  0.450  0.452  0.500   
                   DP+VoT    0.259  0.241  0.550  0.535  0.483  0.484  0.300   
ScanNet   Qwen-7B  ZS        0.383  0.518  0.417  0.523  0.029  0.051  0.800   
                   w/ DP     0.683  0.613  0.733  0.736  0.412  0.484  0.533   
                   DP+CoT    0.467  0.395  0.417  0.311  0.559  0.566  0.467   
                   DP+VoT    0.517  0.482  0.533  0.483  0.471  0.390  0.267   
          Qwen-32B ZS        0.800  0.888  0.700  0.799  0.088  0.153  0.800   
                   w/ DP     0.950  0.949  0.817  0.821  0.294  0.364  0.800   
                   DP+CoT    0.867  0.867  0.667  0.689  0.382  0.472  0.600   
                   DP+VoT    0.897  0.917  0.667  0.674  0.349  0.397  0.800   
          Qwen-72B ZS        0.950  0.966  0.932  0.953  0.294  0.429  0.800   
                   w/ DP     0.983  0.983  0.878  0.888  0.235  0.289  0.778   
                   DP+CoT    0.633  0.637  0.682  0.652  0.368  0.399  0.511   
                   DP+VoT    0.800  0.802  0.784  0.774  0.500  0.493  0.644   
          GPT-4o   ZS        0.917  0.932  0.733  0.735  0.471  0.474  0.867   
                   w/ DP     0.967  0.967  0.700  0.703  0.412  0.323  0.933   
                   DP+CoT    0.483  0.456  0.650  0.621  0.441  0.428  0.400   
                   DP+VoT    0.450  0.420  0.533  0.500  0.500  0.497  0.467   
ScanNet++ Qwen-7B  ZS        0.117  0.191  0.217  0.294  0.000  0.000  1.000   
                   w/ DP     0.550  0.418  0.600  0.600  0.500  0.667  0.500   
                   DP+CoT    0.483  0.439  0.450  0.344  0.000  0.000  0.500   
                   DP+VoT    0.550  0.508  0.567  0.474  0.000  0.000  0.250   
          Qwen-32B ZS        0.467  0.629  0.583  0.686  0.500  0.667  0.750   
                   w/ DP     0.883  0.897  0.717  0.705  1.000  1.000  0.750   
                   DP+CoT    0.683  0.668  0.633  0.642  0.500  0.667  0.250   
                   DP+VoT    0.783  0.798  0.633  0.653  0.500  0.667  0.500   
          Qwen-72B ZS        0.806  0.891  0.871  0.893  0.000  0.000  0.875   
                   w/ DP     0.951  0.975  0.871  0.873  0.000  0.000  1.000   
                   DP+CoT    0.610  0.610  0.659  0.640  0.333  0.500  0.125   
                   DP+VoT    0.775  0.772  0.631  0.615  0.167  0.286  0.833   
          GPT-4o   ZS        0.900  0.908  0.783  0.790  0.500  0.667  1.000   
                   w/ DP     0.883  0.883  0.683  0.683  0.000  0.000  1.000   
                   DP+CoT    0.517  0.513  0.567  0.552  0.000  0.000  0.750   
                   DP+VoT    

In [89]:
# print latex code
print(swap.to_latex(
    column_format="lll|cc|cc|cc|cc|cc|cc|cc",
    multirow=True,
    multicolumn=True,
    multicolumn_format="c",
    float_format="%.3f",
    escape=False,
    label="tab:c1-with-trap",
    caption="C1 with trap.",
    bold_rows=True,
    position="tb",
))

\begin{table}[tb]
\caption{C1 with trap.}
\label{tab:c1-with-trap}
\begin{tabular}{lll|cc|cc|cc|cc|cc|cc|cc}
\toprule
 &  &  & \multicolumn{2}{c}{Pitch} & \multicolumn{2}{c}{Yaw} & \multicolumn{2}{c}{Roll} & \multicolumn{2}{c}{U/D} & \multicolumn{2}{c}{L/R} & \multicolumn{2}{c}{F/B} & \multicolumn{2}{c}{Avg} \\
 &  &  & Acc. & F1 & Acc. & F1 & Acc. & F1 & Acc. & F1 & Acc. & F1 & Acc. & F1 & Acc. & F1 \\
Dataset & Model & Strategy &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{16}{*}{\textbf{7 Scenes}} & \multirow[t]{4}{*}{\textbf{Qwen-7B}} & \textbf{ZS} & 0.052 & 0.092 & 0.200 & 0.285 & 0.000 & 0.000 & 0.100 & 0.175 & 0.250 & 0.317 & 0.111 & 0.183 & 0.096 & 0.144 \\
\textbf{} & \textbf{} & \textbf{w/ DP} & 0.655 & 0.566 & 0.500 & 0.480 & 0.333 & 0.420 & 0.500 & 0.475 & 0.594 & 0.583 & 0.556 & 0.556 & 0.488 & 0.487 \\
\textbf{} & \textbf{} & \textbf{DP+CoT} & 0.310 & 0.218 & 0.517 & 0.412 & 0.417 & 0.418 & 0.800 & 0.800 & 0.344 & 0.250 & 0.556 & 0.593 & 0.473 & 0.423

# C1 cv methods

In [90]:
data_dir = Path("/home/u5u/kdeng.u5u/spatial-reasoning-of-LMs/result/final-table-cv-methods/single-dof-cls")
c1_cv_method = _read_c1_cv(data_dir)
c1_cv_method = _rename_table(c1_cv_method)

c1_cv_method = _set_factor_as_ordered_cate(c1_cv_method, "Dataset", dataset_order)
c1_cv_method = _set_factor_as_ordered_cate(c1_cv_method, "Model", model_order)
c1_cv_method = _set_factor_as_ordered_cate(c1_cv_method, "DoF", dof_order)

### Save c1 cv methods results

In [91]:
pivot = c1_cv_method.pivot_table(
    index=["Dataset", "Model"],
    columns="DoF",
    values=["Acc.", "F1"],
)
swap = pivot.swaplevel(0, 1, axis=1).sort_index(axis=1)
swap.columns.names = [None, None] # for latex use
swap = _cal_avg_of_acc_f1(swap, count=counting_table).round(3)

swap.to_csv("result/csv-tables/c1-cv-method.csv")
swap

/local/user/1483801110/ipykernel_110261/803635906.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = c1_cv_method.pivot_table(


Pitch           Yaw          Roll           U/D           L/R  \
                 Acc.     F1   Acc.     F1   Acc.     F1   Acc.     F1   Acc.   
Dataset  Model                                                                  
7 Scenes SIFT   1.000  1.000  1.000  1.000  0.983  0.983  0.800  0.840  0.875   
         LoFTR  1.000  1.000  0.983  0.983  0.983  0.983  0.800  0.800  0.969   
ScanNet  SIFT   0.833  0.847  0.900  0.901  0.912  0.940  0.667  0.664  0.688   
         LoFTR  0.967  0.967  1.000  1.000  0.971  0.970  0.667  0.678  0.854   

                       F/B         Avg         
                   F1 Acc.   F1   Acc.     F1  
Dataset  Model                                 
7 Scenes SIFT   0.873  1.0  1.0  0.969  0.973  
         LoFTR  0.969  1.0  1.0  0.967  0.967  
ScanNet  SIFT   0.701  1.0  1.0  0.847  0.854  
         LoFTR  0.865  1.0  1.0  0.939  0.941

In [92]:
# print latex code
print(swap.to_latex(
    column_format="lll|cc|cc|cc|cc|cc|cc|cc",
    multirow=True,
    multicolumn=True,
    multicolumn_format="c",
    float_format="%.3f",
    escape=False,
    label="tab:c1-cv-method",
    caption="C1 by computer vision methods.",
    bold_rows=True,
    position="tb",
))

\begin{table}[tb]
\caption{C1 by computer vision methods.}
\label{tab:c1-cv-method}
\begin{tabular}{lll|cc|cc|cc|cc|cc|cc|cc}
\toprule
 &  & \multicolumn{2}{c}{Pitch} & \multicolumn{2}{c}{Yaw} & \multicolumn{2}{c}{Roll} & \multicolumn{2}{c}{U/D} & \multicolumn{2}{c}{L/R} & \multicolumn{2}{c}{F/B} & \multicolumn{2}{c}{Avg} \\
 &  & Acc. & F1 & Acc. & F1 & Acc. & F1 & Acc. & F1 & Acc. & F1 & Acc. & F1 & Acc. & F1 \\
Dataset & Model &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{2}{*}{\textbf{7 Scenes}} & \textbf{SIFT} & 1.000 & 1.000 & 1.000 & 1.000 & 0.983 & 0.983 & 0.800 & 0.840 & 0.875 & 0.873 & 1.000 & 1.000 & 0.969 & 0.973 \\
\textbf{} & \textbf{LoFTR} & 1.000 & 1.000 & 0.983 & 0.983 & 0.983 & 0.983 & 0.800 & 0.800 & 0.969 & 0.969 & 1.000 & 1.000 & 0.967 & 0.967 \\
\cline{1-16}
\multirow[t]{2}{*}{\textbf{ScanNet}} & \textbf{SIFT} & 0.833 & 0.847 & 0.900 & 0.901 & 0.912 & 0.940 & 0.667 & 0.664 & 0.688 & 0.701 & 1.000 & 1.000 & 0.847 & 0.854 \\
\textbf{} & \textbf{